In [1]:
import urllib.request
import os

#url="http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.xls"
filepath="C:/Users/USER/PythonWork/titanic3.xls"
if not os.path.isfile(filepath):
    result = urllib.request.urlretrieve(filepath)
    #result = urllib.request.urlretrieve(url,filepath)
    print('download:',result)

In [2]:
import numpy
import pandas as pd

In [3]:
#使用pandas提供的read_excel()方法讀取titanic3.xls
all_df = pd.read_excel(filepath)

In [4]:
#查看前2筆資料
all_df[:2]

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"


欄位        欄位說明        資料說明
survival    是否生存        0=否，1=是
pclass      艙等            1=頭等艙，2=二等艙，3=三等艙
name        姓名
sex         性別
age         年齡
sibsp       手足或配偶也在船上數量
parch       雙親或子女也在船上數量
ticket      車票號碼
fare        旅客費用
cabin       艙位號碼
embarked    登船港口

In [5]:
#ticket、cabin與預測結果關聯不大所以忽略
cols=['survived','name','pclass','sex','age','sibsp','parch','fare','embarked']
all_df = all_df[cols]

In [6]:
all_df[:2]

,survived,name,pclass,sex,age,sibsp,parch,fare,embarked
0,1,"Allen, Miss. Elisabeth Walton",1,female,29.0000,0,0,211.3375,S
1,1,"Allison, Master. Hudson Trevor",1,male,0.9167,1,2,151.5500,S


姓名欄位後續在預測階段會使用，但是訓練時不需要，必須先移除。
有幾筆資料age欄位是null值，所以必須將null值改為平均值
有幾筆資料fare欄位是null值，所以必須將null值改為平均值
性別欄位是文字，我們必須轉換為0與1
分類特徵欄位有 3個分類C、Q、S，必須使用 Onehot encoding 轉換

In [7]:
#將name欄位移除
df = all_df.drop(['name'],axis=1)

In [8]:
#找出欄位含有null值的
all_df.isnull().sum()

survived      0
name          0
pclass        0
sex           0
age         263
sibsp         0
parch         0
fare          1
embarked      2
dtype: int64

In [9]:
#age欄位null值填上資料平均值
#使用df['age'].mean()方法，計算age欄位的平均值，然後再使用 df['age'].fillna(age_mean)，將null填入平均值。
age_mean = df['age'].mean()
df['age'] =  df['age'].fillna(age_mean)

#fare欄位null值填上資料平均值
fare_mean = df['fare'].mean()
df['fare'] = df['fare'].fillna(age_mean)

#轉換性別欄位1與0
#使用map方法，將'female'轉為0、'male'轉為1
df['sex'] = df['sex'].map({'female':0,'male':1}).astype(int)

#將embarked欄位以onehot encoding轉換
#使用get_dummies()傳入參數data,columns
x_OneHot_df = pd.get_dummies(data=df,columns=["embarked"])

In [10]:
#查看轉換後的data frame
x_OneHot_df[:2]

,survived,pclass,sex,age,sibsp,parch,fare,embarked_C,embarked_Q,embarked_S
0,1,1,0,29.0000,0,0,211.3375,0,0,1
1,1,1,1,0.9167,1,2,151.5500,0,0,1


In [11]:
#dataframe 轉換為array
ndarray = x_OneHot_df.values

In [12]:
ndarray.shape

(1309, 10)

In [13]:
#第0個欄位是label，第1個欄位及之後的欄位是features。
ndarray[:2]

array([[  1.    ,   1.    ,   0.    ,  29.    ,   0.    ,   0.    ,
        211.3375,   0.    ,   0.    ,   1.    ],
       [  1.    ,   1.    ,   1.    ,   0.9167,   1.    ,   2.    ,
        151.55  ,   0.    ,   0.    ,   1.    ]])

In [14]:
#第1維度是筆數 第2維度是欄位
#:等於擷取所有筆數 第0欄位是標籤欄位
Label = ndarray[:,0]
#:等於擷取所有筆數 1: 是特徵欄位
Features = ndarray[:,1:]

In [15]:
Label[:2]


array([1., 1.])

In [16]:
Features[:2]

array([[  1.    ,   0.    ,  29.    ,   0.    ,   0.    , 211.3375,
          0.    ,   0.    ,   1.    ],
       [  1.    ,   1.    ,   0.9167,   1.    ,   2.    , 151.55  ,
          0.    ,   0.    ,   1.    ]])

以上執行結果，因為數值特徵欄位單位不同，例如年齡29歲、運費211元等，數字差異很大，沒有共同標準。這時就要使用標準化，讓所有數值都在0、1之間。

In [17]:
#將ndarray特徵欄位進行標準化
#匯入sklearn的資料預處理模組
from sklearn import preprocessing

In [18]:
#使用preprocessing.MinMaxScaler來進行標準化 標準化範圍在0-1之間
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0,1))

In [20]:
#使用minmax_scale.fit_transfrom進行標準化
scaledFeatures = minmax_scale.fit_transform(Features)

In [21]:
#查看前2筆資料
scaledFeatures[:2]

array([[0.        , 0.        , 0.36116884, 0.        , 0.        ,
        0.41250333, 0.        , 0.        , 1.        ],
       [0.        , 1.        , 0.00939458, 0.125     , 0.22222222,
        0.2958059 , 0.        , 0.        , 1.        ]])

In [22]:
#將資料以隨機方式分為訓練資料及測試資料
#依照8:2的比例使用numpy.random.rand產生msk
msk = numpy.random.rand(len(all_df)) < 0.8

train_df = all_df[msk] #產生訓練資料，為全部資料的80%
test_df = all_df[~msk] #產生測試資料，為全部資料的20%

In [23]:
print('total:',len(all_df),
      'train:',len(train_df),
      'test:',len(test_df)
     )

total: 1309 train: 1066 test: 243


In [32]:
#建立PreprocessData函數進行資料的預處理
def PreprocessData(raw_df):
    df=raw_df.drop(['name'], axis=1)
    age_mean = df['age'].mean()
    df['age'] = df['age'].fillna(age_mean)
    fare_mean = df['fare'].mean()
    df['fare'] = df['fare'].fillna(fare_mean)
    df['sex']= df['sex'].map({'female':0, 'male': 1}).astype(int)
    x_OneHot_df = pd.get_dummies(data=df,columns=["embarked" ])

    ndarray = x_OneHot_df.values
    Features = ndarray[:,1:]
    Label = ndarray[:,0]

    minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))
    scaledFeatures=minmax_scale.fit_transform(Features)    
    
    return scaledFeatures,Label

In [33]:
#將訓練資料與測試資料進行預處理
#train_Features訓練資料的特徵欄位、train_Label訓練資料的標籤欄位
#test_Features測試資料的特徵欄位、test_Label測試資料的標籤欄位
train_Features,train_Label=PreprocessData(train_df)
test_Features,test_Label=PreprocessData(test_df)

In [34]:
train_Features[:2]

array([[0.        , 0.        , 0.36116884, 0.        , 0.        ,
        0.41250333, 0.        , 0.        , 1.        ],
       [0.        , 0.        , 0.0229641 , 0.125     , 0.22222222,
        0.2958059 , 0.        , 0.        , 1.        ]])

In [35]:
train_Label[:2]

array([1., 0.])